# Detect car damage from video

In this notebook I will show how to create frames from a given video, classify each frame using the pretrained classifier, annotate each frame acoordingly, and finally make a video from those annotated frames. This would be useful for damage assessment, where a user would feed a video of a damaged car as input, and the damage location/extent would be predicted instantly. This can be very beneficial for insurance companies as well as the car owners. 

In [1]:
%reload_ext autoreload
%autoreload 2

In [23]:
# importing libraries
#! pip install pytube
from pytube import YouTube
#! conda install -c conda-forge imageio
import imageio
#! pip install moviepy
from moviepy.editor import *
from ipywidgets import Video
import os
import pandas as pd
import shutil
from fastai import *
from fastai.vision import *
from fastai.widgets import *
from PIL import Image
import hashlib
#import scipy.misc
#from scipy.misc import imread, imresize, imshow
import time
import numpy as np
from shutil import copyfile
import glob
import matplotlib.pyplot as plt
import cv2
from os.path import isfile, join

## Download youtube video

In [9]:
filepath = 'C:/Users/manas/Desktop/Kaggle projects/Video_detection'
url = 'https://www.youtube.com/watch?v=ETtwBWe9yQc'
save_fn = 'Damaged_car1'
yt = YouTube(url).streams.first().download(output_path=filepath, filename=save_fn)

## Read Video, save to images and annotate them

### Methodology
I have combined 2 models from previous training: 1st model- to differentiate between undamaged cars/out of domain images from damaged cars; 2nd model- to predict damage location. I have broadly classified into 3 damage classes, bonnet, sideways and rear. Definitely the result would imrove with larger dataset and more specific damage location. Here, the dataset is first classified with 1st learner. When any damage is detected then the image goes through the 2nd classifier and the damage type is annotated on the respective frame. A frame without damage is not annotated.

In [4]:
vidcap = cv2.VideoCapture('C:/Users/manas/Desktop/Kaggle projects/Video_detection/Damaged_car1.mp4')

In [5]:
frame_folder_path = 'C:/Users/manas/Desktop/Kaggle projects/Video_detection/Damaged_car_1_frames/'

In [6]:
# loading learner objects
learn_car = load_learner('C:/Users/manas/Desktop/Kaggle projects/cars')
learn_dmg = load_learner('C:/Users/manas/Desktop/Kaggle projects/Car crash')

In [7]:
def dmg_detect(learn, frame_path):
    prediction = learn.predict(open_image(frame_path))
    category = prediction[0]
    probability = "{0:.4f}".format(max(prediction[2]))
    return category, probability    

In [ ]:
filepath = 'C:/Users/manas/Desktop/Kaggle projects/Video_detection/Damaged_car_1_frames_text/'
count = 0
success = 1
while success:
    success,frame = vidcap.read()
    frame_path = frame_folder_path + f'frame{count}.jpg'
    cv2.imwrite(frame_path,frame)
    #print(frame_path)
    
    # model prediction: damaged car
    category_car, probability_car = dmg_detect(learn_car,frame_path)
    if str(category_car) =='cars_undamaged':
        category_damage = ''
    else:
        category_dmg, probability_dmg = dmg_detect(learn_dmg,frame_path)
        category_damage = 'Damaged car: ' + str(category_dmg)
        
     
     
    # writing category text on image 
    
    if category_damage != '':
        #text = f'{category} \n {probability}'
        text = f'{category_damage}'

        #Writing on iamge
        font = cv2.FONT_HERSHEY_SIMPLEX

        # set the rectangle background to white
        rectangle_bgr = (255, 255, 255)
        # get the width and height of the text box
        (text_width, text_height) = cv2.getTextSize(text, font, fontScale=1.0, thickness=1)[0]
        # set the text start position
        text_offset_x = 5
        text_offset_y = 16
        # make the coords of the box with a small padding (change  coordinate to adjust)
        box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 10, text_offset_y + text_height + 10))
        cv2.rectangle(frame, box_coords[0], box_coords[1], rectangle_bgr, cv2.FILLED)

        
        img_with_text = cv2.putText(frame,text,(10, 40),
            font,1,(255,0,0),2,cv2.LINE_AA)

        # Save the image

        cv2.imwrite(filepath + f'frame{count}.jpg',img_with_text)

    else:
        cv2.imwrite(filepath + f'frame{count}.jpg',frame)

    cv2.imshow('frame',frame)
    cv2.waitKey(1)
    count+=1

The model sometimes confuses between bonnet and trunk due to their similar appearance. I need to train the model with even larger and carefully curated dataset to make the it more efficient.

## From image to video

In [27]:
pathIn= 'C:/Users/manas/Desktop/Kaggle projects/Video_detection/Damaged_car_1_frame_text_1/'
pathOut = 'C:/Users/manas/Desktop/Kaggle projects/Video_detection/damaged_car_text.mp4'
fps = 20
frame_array = []
files = [f for f in os.listdir(pathIn)]

for i in range(len(files)):
    filename=pathIn + 'frame'+' ('+ str(i+1)+ ')'+ '.jpg'
    
    #reading each files
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)
    
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()


## Discussion and future scope of work

Overall, the model did pretty good job in detecting damaged car, and its damage location. But there are still rooms for improvements.

As noted earlier, I need to more data for each class to make the model more efifcient. This is  just a prototype. But to make model more useful, the structure of the model would be as follows.

classifier 1:
classes: out of domain, cars

if class== 'cars',
classifier 2:
classes: undamaged, damaged

if class == 'damaged',
classifier 3:
classes: bonnet, bumper, windshield, sideways, trunk, tyre etc.

for each class,
classifier 4:
classes: major, minor